In [21]:
from ete3 import Tree as eTree
from dendropy import Tree
import pandas as pd
'''
For the tree analysis, I am thinking that we could 
measure precision and recall as well! The way to do 
this is to collapse all the branches which are below a 
certain support value threshold (say, 0.9). 
Precision is given by [#edges in inferred trees that are also in reference tree / #edges in inferred trees] and 
Recall is given by [#edges in inferred trees that are also in reference tree / #edges in reference trees]'''

bootstrap_threshold = 100.0
bootstrap_threshold_ref = 100.0

In [22]:
def trans(one_zero_list):
    return ''.join(['1' if '0' in i else '0' for i in list(one_zero_list)])

In [23]:
tree_data = pd.read_csv('/Users/daviddylus/Downloads/benchmark_reads - like_shen.csv')
tree_data['precision'] = None
tree_data['recall'] = None

In [24]:
# make sure that I check for the complement for it
# check also edge cases

for i,r in tree_data.iterrows():
    if 'ont' in r.technology[0:3].lower():
        techh = 'nan'
    else:
        techh = r.technology[0:3].lower()
        
    if 'arath' in r['species']:
        r2t_nwk_str = r['r2t tree shen'].replace(str(r.dist)+'_'+techh,'ARATH').replace('HORVV','HORVD')
    if 'mouse' in r['species']:
        r2t_nwk_str = r['r2t tree shen'].replace(str(r.dist)+'_'+techh,'MOUSE')
    if 'yeast' in r['species']:
        r2t_nwk_str = r['r2t tree shen'].replace(str(r.dist)+'_'+techh,'YEAST').replace('SLARO', 'SALR5').replace('CAPOW', 'CAPO3')
    
    if 'YEAST' in r2t_nwk_str or 'MOUSE' in r2t_nwk_str or 'ARATH' in r2t_nwk_str:
        r2t_tree = eTree(r2t_nwk_str)# .get(data="((A,B),(C,D));", schema="newick")
        ref_tree = eTree(r['ref trees'])
        r2t_matrix = {i:[] for i in sorted(r2t_tree.get_leaf_names())}
        ref_matrix = {i:[] for i in sorted(ref_tree.get_leaf_names())}

        for node in r2t_tree.traverse("postorder"):
    #       Do some analysis on node
            if len(node.get_leaf_names()) > 1 and len(node.get_leaf_names()) < len(r2t_tree.get_leaf_names()):
    #             print(node.support)
        #         print(node.get_leaf_names())
                if node.support >= bootstrap_threshold:
                    for k,v in r2t_matrix.items():
                        if k in node.get_leaf_names():
                            v.append('1')
                        else:
                            v.append('0')
    #             else:
    #                 print(node.support)
        r2t_df = pd.DataFrame(r2t_matrix)

        for node in ref_tree.traverse("postorder"):
          # Do some analysis on node
            if len(node.get_leaf_names()) > 1 and len(node.get_leaf_names()) < len(ref_tree.get_leaf_names()):
        #         print(node.support)
        #         print(node.get_leaf_names())
                if node.support >= bootstrap_threshold_ref:
                    for k,v in ref_matrix.items():
                        if k in node.get_leaf_names():
                            v.append('1')
                        else:
                            v.append('0')
        ref_df = pd.DataFrame(ref_matrix)

        ref_str = [''.join(list(r)) for i,r in ref_df.iterrows()]
        r2t_str = [''.join(list(r)) for i,r in r2t_df.iterrows()]
        if r2t_str:
            edges_in_r2t = 0
            for r2t_i in r2t_str:
                if r2t_i in ref_str or trans(r2t_i) in ref_str:
                    edges_in_r2t = edges_in_r2t + 1

            tree_data.at[i, 'precision'] = edges_in_r2t / len(r2t_str)
            tree_data.at[i, 'recall'] = edges_in_r2t / len(ref_str)
#         else:
#             tree_data.at[i, 'precision'] = 0.0
#             tree_data.at[i, 'recall'] = 0.0
            
    else:
        print(r2t_nwk_str, r.dist, r.coverage)

(SELML:0.2017112724,((CHLVA:0.3319658251,KLEFL:0.2025275205)100:0.1001503948,PHYPA:0.1692402352)100:0.0404629820,(((((((SORBI:0.0144119883,MAIZE:0.0233175158)100:0.0167711769,SETIT:0.0234941358)100:0.0148894922,ERATE:0.0436059071)100:0.0161240577,(((((TRIUA:0.0133240262,AEGTA:0.0114795345)96:0.0020026112,WHEAT:0.0019900860)100:0.0072323361,HORVD:0.0111089346)100:0.0307235857,BRADI:0.0302675791)100:0.0340880235,(ORYPU:0.0092538716,(ORYLO:0.0129958718,(ORYNI:0.0018073128,ORYRU:0.0019653642)100:0.0026174009)100:0.0059106855)100:0.0390859952)100:0.0105142377)100:0.1155420735,(MUSAM:0.0000012278,MUSAC:0.0000012278)100:0.1040472257)100:0.0510950961,(SOLLC:0.0099731128,SOLTU:0.0093855109)100:0.1412780972)100:0.0391178268,AMBTC:0.1253000367)100:0.1443034005); 6 0.5
(SORBI:0.0143772183,(((((((SELML:0.2020734261,((KLEFL:0.2033598320,CHLVA:0.3259499568)100:0.0999386083,PHYPA:0.1690281462)100:0.0403138859)100:0.1440575002,AMBTC:0.1251203189)100:0.0391876111,(SOLLC:0.0099607513,SOLTU:0.0093864921)1

In [25]:
tree_data.to_csv('/Users/daviddylus/Desktop/precision_recall_100.csv')

In [15]:
tree_data


,species,coverage,dist,assembly_time,technology,assembly tree,r2t tree shen,r2t tree,ref trees,precision,recall
0,arath,0.2,0,NaN,Illumina,NaN,"(CHLVA:0.3296090866,((SELML:0.1971417168,(((((...","(BRARP:0.00539097,((((((((((((((TRIUA:0.019465...","(VITVI:0.09177319336826902,(((((((ORYLO:0.0175...",1.0,0.966667
1,arath,0.2,1,NaN,Illumina,NaN,"(SELML:0.2002209391,((CHLVA:0.3438467287,KLEFL...","(ARATH:0.0345819,((BRARP:0.00537901,(BRANA:0.0...","(VITVI:0.09177761921655042,(((((((ORYLO:0.0175...",1.0,0.931034
2,arath,0.2,2,NaN,Illumina,NaN,"(SORBI:0.0139614912,(((((((((((BRARP:0.0041331...","(TRIUA:0.0194671,(((((((((((((BRARP:0.00533048...","(VITVI:0.09179352450169354,(((((((ORYLO:0.0175...",1.0,0.965517
3,arath,0.2,3,NaN,Illumina,NaN,"(3_ill:0.0704052773,((((CHLVA:0.3267546679,KLE...","(ARATH:0.100639,((((((((((((TRIUA:0.0194609,AE...","(VITVI:0.09141184952100921,(((((((ORYLO:0.0175...",0.869565,0.740741
4,arath,0.2,4,NaN,Illumina,NaN,"(4_ill:0.0654262303,((((CHLVA:0.3459067888,KLE...","(ARATH:0.0983816,(((((((((((TRIUA:0.019467,AEG...","(VITVI:0.08986284816204641,(((((((ORYLO:0.0175...",0.857143,0.72
...,...,...,...,...,...,...,...,...,...,...,...
373,yeast,20.0,2,20774.0,PacBIO,"(SALR5:0.2724975652,(((((((BOTFB:0.1370638124,...","(SPIPN:0.2299730648,(((SALR5:0.3076006667,MONB...","(YEAST:0.151916,(((((SALR5:0.333326,MONBE:0.31...","(SALR5:0.2511666572314431,((((((((TALSN:0.1166...",0.941176,0.888889
374,yeast,20.0,3,20774.0,PacBIO,"(BOTFB:0.1438173545,(((COCLU:0.1828847018,AURP...","(TALSN:0.1347920486,(((((((CAPO3:0.3709647034,...","(MONBE:0.316357,((((YEAST:0.214991,((((TALSN:0...","(SALR5:0.2589124419037486,(((((((TALSN:0.12013...",0.933333,0.875
375,yeast,20.0,4,20774.0,PacBIO,"(YEAST:0.4007114048,(((MONBE:0.2850049111,SALR...","(4_pac:0.1387700228,((((MONBE:0.2974460978,SAL...","(MONBE:0.314583,((((YEAST:0.162328,SCHPO:0.309...","(SALR5:0.29433779029474216,(((YEAST:0.48484088...",1.0,1.0
376,yeast,20.0,5,20774.0,PacBIO,"(SALR5:0.3037354340,(((SPIPN:0.2072047071,BATD...","(MONBE:0.2945619465,(((5_pac:0.1670709379,(BAT...","(CREFR:0.464086,(SALR5:0.333488,MONBE:0.314554...","(SALR5:0.299908542030726,(((YEAST:0.6104814961...",0.75,1.0


In [26]:
r2t_str[1].split()

['10110']

In [27]:
trans(r2t_str[1])

'01001'

In [48]:
tree_data

,species,coverage,dist,assembly_time,technology,assembly tree,r2t tree shen,r2t tree,ref trees,precision,recall
0,arath,0.2,0,NaN,Illumina,NaN,"(CHLVA:0.3296090866,((SELML:0.1971417168,(((((...","(BRARP:0.00539097,((((((((((((((TRIUA:0.019465...","(VITVI:0.09177319336826902,(((((((ORYLO:0.0175...",1.000000,0.906250
1,arath,0.2,1,NaN,Illumina,NaN,"(SELML:0.2002209391,((CHLVA:0.3438467287,KLEFL...","(ARATH:0.0345819,((BRARP:0.00537901,(BRANA:0.0...","(VITVI:0.09177761921655042,(((((((ORYLO:0.0175...",1.000000,0.870968
2,arath,0.2,2,NaN,Illumina,NaN,"(SORBI:0.0139614912,(((((((((((BRARP:0.0041331...","(TRIUA:0.0194671,(((((((((((((BRARP:0.00533048...","(VITVI:0.09179352450169354,(((((((ORYLO:0.0175...",1.000000,0.933333
3,arath,0.2,3,NaN,Illumina,NaN,"(3_ill:0.0704052773,((((CHLVA:0.3267546679,KLE...","(ARATH:0.100639,((((((((((((TRIUA:0.0194609,AE...","(VITVI:0.09141184952100921,(((((((ORYLO:0.0175...",0.869565,0.740741
4,arath,0.2,4,NaN,Illumina,NaN,"(4_ill:0.0654262303,((((CHLVA:0.3459067888,KLE...","(ARATH:0.0983816,(((((((((((TRIUA:0.019467,AEG...","(VITVI:0.08986284816204641,(((((((ORYLO:0.0175...",0.857143,0.720000
...,...,...,...,...,...,...,...,...,...,...,...
373,yeast,20.0,2,20774.0,PacBIO,"(SALR5:0.2724975652,(((((((BOTFB:0.1370638124,...","(SPIPN:0.2299730648,(((SALR5:0.3076006667,MONB...","(YEAST:0.151916,(((((SALR5:0.333326,MONBE:0.31...","(SALR5:0.2511666572314431,((((((((TALSN:0.1166...",0.941176,0.888889
374,yeast,20.0,3,20774.0,PacBIO,"(BOTFB:0.1438173545,(((COCLU:0.1828847018,AURP...","(TALSN:0.1347920486,(((((((CAPO3:0.3709647034,...","(MONBE:0.316357,((((YEAST:0.214991,((((TALSN:0...","(SALR5:0.2589124419037486,(((((((TALSN:0.12013...",0.933333,0.823529
375,yeast,20.0,4,20774.0,PacBIO,"(YEAST:0.4007114048,(((MONBE:0.2850049111,SALR...","(4_pac:0.1387700228,((((MONBE:0.2974460978,SAL...","(MONBE:0.314583,((((YEAST:0.162328,SCHPO:0.309...","(SALR5:0.29433779029474216,(((YEAST:0.48484088...",1.000000,0.800000
376,yeast,20.0,5,20774.0,PacBIO,"(SALR5:0.3037354340,(((SPIPN:0.2072047071,BATD...","(MONBE:0.2945619465,(((5_pac:0.1670709379,(BAT...","(CREFR:0.464086,(SALR5:0.333488,MONBE:0.314554...","(SALR5:0.299908542030726,(((YEAST:0.6104814961...",1.000000,1.000000
